<div class="alert alert-warning">
WARNING: This workflow is still in early stages. The priority is the more general `eeg-viewer` workflow. When further along, the `eeg-viewer` workflow will form the basis of this `mne-raw` workflow.
</div>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')
from neurodatagen.eeg import simulate_eeg

## Generate data

In [ ]:
raw = simulate_eeg(duration=5, n_channels=20, sfreq=1000)

In [ ]:
raw.info

## Visualize EEG

In [ ]:
def view_eeg(raw, spacing=2):
    """
    Plot stacked EEG traced

    Parameters:
        raw (mne.io.RawArray): The EEG data as a `mne.io.RawArray` object.
        spacing (float): multipler to vertically space traces

    Returns:
        holoviews.core.overlay.NdOverlay: The EEG plot.

    """
    data = raw.get_data()
    ch_names = raw.ch_names

    # Calculate the offset between channels to avoid visual overlap
    offset = np.max(np.abs(data)) * spacing

    # Create a hv.Curve element per chan
    channel_curves = {}
    for i, channel_data in enumerate(data):
        channel_curves[ch_names[i]] = hv.Curve((raw.times, channel_data + (i * offset)), 'Time').opts(color='black', tools=['hover'])

    # Create mapping from yaxis location to ytick for each channel
    yticks = [(i * offset, channel_name) for i, channel_name in enumerate(ch_names)]

    # Create hv overlay of curves
    eeg_viewer = hv.NdOverlay(channel_curves, kdims='Channel').opts(
        width=600, height=600, padding=.01, xlabel='Time', ylabel='Channel', yticks=yticks, show_legend=False)

    return eeg_viewer

In [ ]:
view_eeg(raw)